# 🚀 Sarika AI - Stage 2 Training
## Context Teachers Sequential Distillation

**Author:** Noushad  
**Repository:** [noushad999/sarika-ai](https://github.com/noushad999/sarika-ai)  
**GPU Required:** T4 (16GB VRAM)  
**Duration:** ~4-6 hours  

### What This Does:
- Loads Llama-3.1-8B student model
- Trains with 6 specialized teacher models sequentially
- Applies LoRA (only ~16M trainable parameters)
- Saves checkpoints to Google Drive

### Teachers:
1. **Bengali Culture** - Mistral-7B
2. **Emotional Intelligence** - Qwen2.5-7B
3. **Conversation Flow** - Llama-3.1-8B
4. **Humor** - Gemma-2-9B
5. **Deep Conversations** - Mistral-7B
6. **Crisis Support** - Qwen2.5-7B

## ✅ Step 1: Check GPU

In [ ]:
!nvidia-smi

import torch
print(f"\n{'='*60}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
print(f"{'='*60}")

## 📂 Step 2: Mount Drive & Clone Repository

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to Drive
import os
os.chdir('/content/drive/MyDrive')

# Clone repository (skip if already exists)
if not os.path.exists('sarika-ai'):
    !git clone https://github.com/noushad999/sarika-ai.git
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists")
    %cd sarika-ai
    !git pull origin main
    print("✓ Repository updated")

# Change to project directory
%cd sarika-ai

!ls -la

## 📦 Step 3: Install Dependencies

In [ ]:
# Install required packages
!pip install -q transformers accelerate peft bitsandbytes datasets huggingface_hub scipy

print("✓ Dependencies installed")

## 🔑 Step 4: HuggingFace Login

In [ ]:
# Login to HuggingFace
from huggingface_hub import login
from getpass import getpass

HF_TOKEN = getpass("Enter your HuggingFace token: ")
login(token=HF_TOKEN)

# Set environment variable
import os
os.environ["HF_TOKEN"] = HF_TOKEN

print("✓ Logged in to HuggingFace")

## ⚙️ Step 5: Update Config for Colab

In [ ]:
%%writefile ml/config.py
"""
Sarika AI - Configuration (Colab Optimized)
"""

import torch
from pathlib import Path
import os

# Paths - Colab
PROJECT_ROOT = Path("/content/drive/MyDrive/sarika-ai")
CHECKPOINT_DIR = PROJECT_ROOT / "ml" / "checkpoints"
MODEL_DIR = PROJECT_ROOT / "models"
DATA_DIR = PROJECT_ROOT / "data"
HF_HOME = str(PROJECT_ROOT / "models" / "cache")

# Create directories
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Set environment
os.environ["HF_HOME"] = HF_HOME
os.environ["TRANSFORMERS_CACHE"] = HF_HOME
os.environ["HF_DATASETS_CACHE"] = str(DATA_DIR / "datasets_cache")

# Device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
GPU_MEMORY_GB = 16  # T4

# Tokens
HF_TOKEN = os.getenv("HF_TOKEN")

# Training Configuration
class TrainingConfig:
    # LoRA parameters
    LORA_R = 16
    LORA_ALPHA = 32
    LORA_DROPOUT = 0.05
    LORA_TARGET_MODULES = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]
    
    # Training hyperparameters
    LEARNING_RATE = 2e-4
    WEIGHT_DECAY = 0.01
    MAX_GRAD_NORM = 1.0
    MAX_SEQ_LENGTH = 512
    BATCH_SIZE = 1
    
    # Distillation
    DISTILLATION_ALPHA = 0.5
    TEMPERATURE = 2.0

class LogConfig:
    LOG_STEPS = 10
    SAVE_STEPS = 100

class SpaceConfig:
    MAX_TOTAL_USAGE = 50
    CLEANUP_THRESHOLD = 80
    MAX_CHECKPOINTS = 3

print(f"✓ Config loaded")
print(f"  Device: {DEVICE}")
print(f"  Project: {PROJECT_ROOT}")

## 🚀 Step 6: Start Training!

In [ ]:
# Run Stage 2 training
!python ml/training/stage2_context_teachers.py

## 📊 Monitor Progress

In [ ]:
# Check GPU usage
!nvidia-smi

In [ ]:
# View checkpoints
!ls -lh ml/checkpoints/stage2_context_teachers/

In [ ]:
# Check disk usage
!df -h | grep drive

## 💾 Download Final Model (Optional)

In [ ]:
# Zip and download final model
!cd ml/checkpoints/stage2_context_teachers && tar -czf final_model.tar.gz final_model/

from google.colab import files
files.download('ml/checkpoints/stage2_context_teachers/final_model.tar.gz')

## 🔄 Update Code (If Needed)

In [ ]:
# Pull latest changes from GitHub
%cd /content/drive/MyDrive/sarika-ai
!git pull origin main

## 🛠️ Troubleshooting

### Out of Memory?
```python
# Reduce batch size in config
# Edit ml/config.py: BATCH_SIZE = 1
```

### Session Timeout?
```python
# Training resumes from last checkpoint automatically
# Just re-run the training cell
```

### Model Download Failed?
```python
# Check HuggingFace token
# Make sure you accepted Llama license: https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct
```